In [96]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from random import random, seed 
from numba import jit

from sklearn import linear_model
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_log_error, mean_absolute_error
from sklearn.model_selection import train_test_split

In [101]:
x = np.arange(0, 1, 0.07)
y = np.arange(0, 1, 0.07)


In [102]:
def createDataPoints(x,y):
    x_d, y_d = np.meshgrid(x,y)
    z_d = frankeFunction(x_d, y_d)
    return x_d, y_d, x_d

In [103]:
def frankeFunction(x,y):
    #noise = np.random.normal(0.5,1,len(x))
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4 #+ noise

def createDataPoints(x,y):
    x_d, y_d = np.meshgrid(x,y)
    z_d = frankeFunction(x_d,y_d)
    return x_d, y_d, z_d

def convertDataPoints(x,y,z):
    x_d = np.ravel(x)
    y_d = np.ravel(y)
    z_d = np.ravel(z)
    return x_d, y_d, z_d

@jit
def create_design_matrix(x, y, n):
#    if len(x_d.shape) > 1:
#        x = np.ravel(x_d)
#        y = np.ravel(y_d)

    N = len(x)
    p = int((n+1)*(n+2)/2)
    X = np.ones((N,p))

    for i in range(1, n+1):
        q = int(i*(i+1)/2)
        for j in range(i+1):
            X[:,q+j] = (x**(i-j))*(y**j)
    return X

def fitBeta(X, z_data):
    beta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(z_data)
    return beta.T

def MSE(z_data, z_model):
    n = np.size(z_model)
    return np.sum((z_data-z_model)**2)/n

def R2(z_data, z_model):
    n = np.size(z_data)
    return 1 - np.sum((z_data-z_model)**2)/np.sum((z_data-(np.sum(z_data)/n))**2)

def trainData(x,z):
    X_train, X_test, z_train, z_test = train_test_split(x,z,test_size=0.3)
    Beta = np.linalg.inv(X_train.T.dot(X_train)).dot(X_train.T).dot(z_train)
    ztilde = X_train @ Beta
    return ztilde
    


In [104]:
x, y, z = createDataPoints(x,y)
x_d, y_d, z_d = convertDataPoints(x,y,z)

In [105]:
X = create_design_matrix(x_d,y_d,3)

In [106]:
beta = fitBeta(X,z_d)

In [107]:
beta

array([ 0.89311875, -0.04010161,  1.39020068, -2.10518019,  1.26897194,
       -6.19995987,  1.20387626,  0.62450989, -1.12002714,  4.27417406])

In [108]:
z_tilde = trainData(X,z_d)

In [110]:
z_tilde

array([ 0.73691305,  0.01836573,  0.12984096,  0.08250559,  0.34768136,
        0.03739574,  0.96357307,  0.85978497,  0.1890093 ,  0.19718881,
        0.48325123,  0.17362676,  0.46067406,  0.93846512,  0.42105139,
        0.15219943,  0.34289176,  0.60440647,  0.90796601,  0.2177614 ,
        0.71413234,  0.78600495,  0.25328577,  0.19985037,  0.4227311 ,
        0.18939124,  0.759885  ,  0.27797901,  0.40329737,  0.59137244,
        0.07288484,  0.15979685,  0.27825085,  0.3851131 ,  0.318169  ,
        0.26080452,  0.50398492,  0.15491801,  0.41097331,  0.63267787,
        0.67545386,  0.29461419,  0.19513508,  0.12470588,  0.11039798,
        0.38844548,  0.25917688,  0.10381253,  0.31501267,  0.25579937,
        0.32426168,  0.62459171,  0.94155829,  0.24918517,  0.36221815,
        0.29599212,  0.06639201,  0.49914688,  0.48708592,  0.34618029,
        0.30760097,  0.8065518 ,  0.81225206,  0.66737269,  0.3360059 ,
        0.35427305,  0.31738934,  0.82073686,  0.86885578,  0.88

In [111]:
MSE(z_d,z_tilde)

ValueError: operands could not be broadcast together with shapes (225,) (157,) 

In [116]:
len(z_tilde)

157